In [15]:
import warnings
warnings.filterwarnings('ignore')
from math import factorial
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as Axes3d
import time
import pandas as pd
from tqdm import tqdm
import numpy as np
from numpy import matlib
import matplotlib.pyplot as plt
import random as random
import math
from tqdm import tqdm
import numpy as np
from numpy import matlib
import matplotlib.pyplot as plt
import random as random
import math

In [16]:
def deleteOneRepositoryMember(rep , gamma):
    gridindices = [item.gridIndex for item in rep]
    OCells = np.unique(gridindices) # ocupied cells
    N = np.zeros(len(OCells))
    for k in range(len(OCells)):
        N[k] = gridindices.count(OCells[k])
    # selection probablity
    p = [math.exp(gamma*item) for item in N]
    p = np.array(p)/sum(p)

    # select cell index
    sci = roulettewheelSelection(p)
    SelectedCell = OCells[sci]

    #selected Cell members
    selectedCellmembers = [item for item in gridindices if item == SelectedCell]

    selectedmemberindex = np.random.randint(0,len(selectedCellmembers))
    #selectedmember = selectedCellmembers[selectedmemberindex]

    # delete memeber
    #rep[selectedmemberindex] = []
    rep = np.delete(rep, selectedmemberindex)

    return rep.tolist()


In [17]:
def SelectLeader(rep , beta):
    gridindices = [item.gridIndex for item in rep]
    OCells = np.unique(gridindices) # ocupied cells
    N = np.zeros(len(OCells))
    for k in range(len(OCells)):
        N[k] = gridindices.count(OCells[k])
    # selection probablity
    p = [math.exp(-beta*item) for item in N]
    p = np.array(p)/sum(p)

    # select cell index
    sci = roulettewheelSelection(p)
    SelectedCell = OCells[sci]

    #selected Cell members
    selectedCellmembers = [item for item in gridindices if item == SelectedCell]

    selectedmemberindex = np.random.randint(0,len(selectedCellmembers))
    # selectedmember = selectedCellmembers[selectedmemberindex]

    return rep[selectedmemberindex]



In [18]:
def roulettewheelSelection(p):
    r = random.random()
    cumsum = np.cumsum(p)
    y = (cumsum<r)
    x= [i for i in y if i==True]
    return len(x)


In [19]:
def FindGridIndex(particle, grid):
    nObj = len(particle.cost)
    NGrid = len(grid[0].LowerBounds)
    
    particle.gridSubIndex = np.zeros((1,nObj))[0]
    for j in range(nObj):  
        index_in_Dim = len( [item for item in grid[j].UpperBounds if particle.cost[j]>item]) 
        particle.gridSubIndex[j] = index_in_Dim

    particle.gridIndex = particle.gridSubIndex[0]

    for j in range(1,nObj):
        particle.gridIndex = particle.gridIndex 
        particle.gridIndex = NGrid*particle.gridIndex
        particle.gridIndex = particle.gridIndex + particle.gridSubIndex[j]

    return particle

In [20]:
def CreateGrid(pop,nGrid,alpha,nobj):
    costs = [item.cost for item in pop]
    Cmin = np.min(costs,axis=0)
    Cmax = np.max(costs,axis=0)
    deltaC = Cmax - Cmin
    Cmin =  Cmin - alpha*deltaC
    Cmax = Cmax + alpha*deltaC
   
    grid = [GridDim() for p in range(nobj)]
    for i in range(nobj):
       dimValues = np.linspace(Cmin[i],Cmax[i],nGrid+1).tolist()
       grid[i].LowerBounds = [-float('inf')] + dimValues
       grid[i].UpperBounds = dimValues  + [float('inf')]
    return grid

In [21]:
def Dominates(x,y):
    x=np.array(x)
    y=np.array(y)
    x_dominate_y = all(x<=y) and any(x<y)
    return x_dominate_y

def DetermineDomination(pop):
    pop_len= len(pop)
    for i in range(pop_len):
         pop[i].IsDominated = False 

    for i in range(pop_len-1):
        for j in range(i+1,pop_len):
            if Dominates(pop[i].cost,pop[j].cost):
                pop[j].IsDominated = True
            if Dominates(pop[j].cost,pop[i].cost):
                pop[i].IsDominated = True

    return pop


# Injecting MOCP BEGINS

In [22]:
def fixer(VA):
    VA = [int(x) for x in VA]
    for i in range(len(VA)):
      if VA[i] < 0:
        VA[i] = 0
      elif VA[i] > 20:
        VA[i] = 20
      
      
    k = []
    for i in range(len(VA)):
        VA[i] = int(round(VA[i]))
        if VA[i] not in k:
            k.append(VA[i])
        else:
            f = 0
            for q in range(VA[i], 20):
                if q not in k:
                    VA[i] = q
                    k.append(q)
                    f = 1
                    break
            if f == 0:
                for q in reversed(range(0, VA[i])):
                    if q not in k:
                        VA[i] = q
                        k.append(q)
                        f = 1
                        break
    return VA

In [28]:
import pandas as pd
import numpy as np
import numpy
def uniform(low, up, size=None):
  return random.sample(range(varMin,varMax),nVar)
import autograd.numpy as anp 
def MOCPParticle(nVar): #passing VARS=init pop and NDIM=number of decision variables into the funct.
        #print(type(VARS))
        #if isinstance(nVar,np.ndarray):
           # nVar = nVar.tolist()
        #if type(nVar) is numpy.ndarray:
        if not isinstance(nVar, list):
            nVar = nVar.tolist()
            nVar = [int(x) for x in nVar]
       
       
        #nVar = list(nVar)
        #if isinstance(nVar,np.ndarray):
         #   nVar = type(nVar)
          #  return 0
                
        #print("Details of X below")

        #VARS.apply(pd.to_numeric)
        #VARS.astype('int32').dtypes
        #df = VARS.convert_objects(convert_numeric=True)
        dataset = pd.read_csv('DistanceMatrixnan2.csv',index_col=0)
        global x
        #x = pd.DataFrame(nVar)
        
        li = []
        cont_distance =[]
        cont_distanc =[]
        ctyu = []
        sam = 0
        a = 0
        sm = []
        city = dataset.columns.to_list()
        #print("city is \n", city)
        v = len(dataset)
        #print("length of dataset is ", v)
        minL = []
        sumL = []
        maxC2C = []
        avgC2C = []
        C2C = []
        end_values = []
        
        r = nVar
        print (r)
        for i in range(len(r)):
            ctyu.append(city[r[i]])
          #print('city being appended is ',city[r[i]])

        
          #print('selected', city[r[i]]  )
        a = i
        dty = dataset[ctyu].iloc[0]

          #print('tobi is ',dty)
        cont_distance = max(dty)
          
        ydt = dataset[ctyu]
        lo = ydt.iloc[0]
        #print(lo)
        for i in range(len(lo)):
            si = 0
            a = i
              
            for j in range(i +1,len(lo), 1):
                si = lo[i] + lo[j]
                sm.append(si)
            t = 0
            for u in sm:
              t = t + u
            #print(t)
            C2 = (t)/(math.comb(len(r), 2))
            print(C2)
        new_dataset = ydt.dropna() 
        
        row_list = []
        new_list =[]
        ij = []
        values = pd.DataFrame()
        for index, row in ydt.iterrows():
            #print('index is length ',len(row))
            for k in range(len(row)):
              
              if isinstance(row.loc[ctyu[k]], pd.core.series.Series):
                
                row_list.append(row.loc[ctyu[k]].iloc[0])
              else:
                row_list.append(row.loc[ctyu[k]])
            #print("row_list\n", row_list)
            for i in row_list:
              if i != min(row_list):
                    #print("I is not equal to min of rowlist")
                    new_list.append(0)
              else:
                    new_list.append(1)
            #print("New list is \n",new_list)
            to_load = pd.Series(new_list, index=ctyu)
            #print("loaded list is \n", to_load)
            values = values.append(to_load, ignore_index=True)
            #print("values \n", values)
            row_list =[]
            new_list =[]

            result = values.apply(pd.value_counts).drop([0])
            #print("result\n", result,"\nMAX", result.max(axis = 1), "\nMIN", result.min(axis = 1))
            end_val = (result.max(axis = 1) - result.min(axis = 1))/result.max(axis = 1)
            
            #print("end_val\n",end_val)
            end_values.append(end_val)

            
            
            
            C2C.append(C2)
            maxC2C.append(cont_distance)
            avgC2C.append(C2)

            for i in r:
              li.append(city[i])
            #print("A new row ") 
            #print(li)
            new_dataset2 = dataset[li]  
            new_dataset2 = new_dataset2.dropna() 
            new_dataset2 = new_dataset2.reset_index(drop=True)
            new_dataset2 =  pd.DataFrame(new_dataset2)
            mins =  new_dataset2.min(axis=1)
            sum1 = 1/(abs(len(new_dataset2))) * np.sum(mins)
            #print(new_dataset2)
            minL.append(mins)
            sumL.append(sum1)
        minDataset = pd.DataFrame(minL)
        maxC2CDataset = pd.DataFrame(maxC2C)
        C2CDataset = pd.DataFrame(C2C)
        EndValDataset = pd.DataFrame(end_values)
        avgC2CDataset = pd.DataFrame(avgC2C)
        
        #norm_EndValDataset = EndValDataset / (EndValDataset.max())
        #print('EndValDataset.max() is \n', EndValDataset.max())
        #print('EndValDataset is \n',norm_EndValDataset)
        #return norm_EndValDataset
        #endvalues_norm = norm_EndValDataset[1.0].to_list()
        
        maxNu = minDataset.max(axis=1) #generates the maximum of minimum
        maxIjogbon = maxNu.values.tolist()
        end_valmod = end_val.values.tolist()
        #print("Maximum of Minimum is ")
        #print(maxNu)
        #print("\n\n")
        #print("Sum")
        maxNuDataset = pd.DataFrame(maxNu)
        sumDataset = pd.DataFrame(sumL) #generates the sum
        
        print("maxIjogbon", maxIjogbon[0])
        print("cont_distance",  cont_distance)
        print("C2", C2 )
        #print("end_valmod", end_valmod[0])
        print("sum", sum1)
        
        return numpy.array([maxIjogbon[0], cont_distance, C2, end_valmod[0],  sum1])
    

In [29]:
nVar = 5 # number of decision vars
varMin = 0
varMax = 20
maxIt = 5
nPop = 455
nRep = 50  # size of repository
w = 0.5 # inertia wieght
c1 = 2 # personal learning coefficient
c2 = 2 # global learning coefficient
wdamping = 0.99

In [30]:
import numpy as np
import numpy
beta = 1 # leader selection pressure
gamma = 1 # deletion selection pressure
NoGrid = 5
alpha=0.1 # nerkhe tavarrom grid

# initialization
class Particle:
    position = []
    cost = []
    velocity = []
    best_position = []
    best_cost = []
    IsDominated = []
    gridIndex = []
    gridSubIndex = []

# for each objective a grid items is division of values of objective cost
class GridDim: 
    LowerBounds = []
    UpperBounds = []

global Particles
#Particles = np.matlib.repmat(Particle,nPop,1)
Particles = [Particle() for p in range(nPop)]
Temp_Particles = []
#print (Particles)

for i in range(nPop):
    Temp_Position = np.array(uniform(varMin,varMax,nVar))
    Particles[i].position = Temp_Position
    Temp_Particles.append(Temp_Position)
    #print('position is', Particles[i].position)
    Particles[i].velocity = np.zeros(nVar)
    Particles[i].cost = np.array(MOCPParticle(Particles[i].position))
    # update best personal Best
    Particles[i].best_position = Particles[i].position
    Particles[i].best_cost = Particles[i].cost
    Particles[i].IsDominated = False 

#print (Particles)
#Particles.cost = MOCP(Temp_Particles, NDIM)   
    
Particles = DetermineDomination(Particles)

Repos = [item for item in Particles if item.IsDominated == False ]
#print("Repos\n\n")
#print (Repos[0].cost)
#print (Repos[1].cost)
nObj =len( Repos[0].cost)
grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)

for r in range(len(Repos)):
    Repos[r] = FindGridIndex(Repos[0],grid)
    #print("r is", r)
    #print ("repos r is", r)


[7, 11, 1, 14, 3]
0.31128248049999996
0.5876624803999999
0.6821920059
0.7386159807999999
0.7386159807999999
maxIjogbon 0.488235886
cont_distance 0.669677499
C2 0.7386159807999999
sum 0.13745931314285714
[14, 1, 3, 11, 13]
0.324265632
0.4900002574
0.6058413925
0.7071528563999999
0.7071528563999999
maxIjogbon 0.299491969
cont_distance 0.669677499
C2 0.7071528563999999
sum 0.09777109776190475
[12, 0, 7, 19, 18]
0.32089424600000005
0.44370780080000005
0.6087308507
0.6893349104000001
0.6893349104000001
maxIjogbon 0.357032701
cont_distance 0.495201728
C2 0.6893349104000001
sum 0.12514888414285716
[18, 3, 19, 5, 9]
0.2828525857
0.4247286818
0.5865377906
0.6648918604
0.6648918604
maxIjogbon 0.268416266
cont_distance 0.508226398
C2 0.6648918604
sum 0.10812156814285714
[1, 10, 18, 8, 19]
0.24425274479999998
0.533788953
0.6799598545000001
0.7483776756
0.7483776756
maxIjogbon 0.365543138
cont_distance 0.607472823
C2 0.7483776756
sum 0.10906521114285714
[5, 10, 18, 16, 7]
0.3162449034
0.6438586514


maxIjogbon 0.357032701
cont_distance 0.607472823
C2 0.9004635312000001
sum 0.1355109939047619
[2, 15, 7, 9, 1]
0.22812159939999996
0.5074920704
0.6617864756999999
0.7316618908
0.7316618908
maxIjogbon 0.44158265
cont_distance 0.557618536
C2 0.7316618908
sum 0.1259551160952381
[15, 14, 17, 7, 9]
0.40353390720000004
0.6823386786000001
0.8519612507
0.9449933855999999
0.9449933855999999
maxIjogbon 0.558930634
cont_distance 0.557618536
C2 0.9449933855999999
sum 0.17462028047619046
[6, 15, 1, 9, 0]
0.2326440459
0.46980502179999994
0.5587332121999999
0.609555852
0.609555852
maxIjogbon 0.495201728
cont_distance 0.557618536
C2 0.609555852
sum 0.14412370295238094
[16, 15, 6, 13, 4]
0.40718974919999995
0.6687597424
0.7897958691
0.8573286072
0.8573286072
maxIjogbon 0.423034124
cont_distance 0.642858658
C2 0.8573286072
sum 0.12122041757142858
[0, 11, 17, 2, 5]
0.1478631074
0.4296617146
0.5488522907
0.5914524296
0.5914524296
maxIjogbon 0.44158265
cont_distance 0.669677499
C2 0.5914524296
sum 0.138921

maxIjogbon 0.446606371
cont_distance 0.417275195
C2 0.4592760148
sum 0.13062986557142858
[16, 8, 6, 19, 15]
0.40807483219999996
0.6123868044
0.7633795661
0.8608689392000001
0.8608689392000001
maxIjogbon 0.403059166
cont_distance 0.642858658
C2 0.8608689392000001
sum 0.132681359
[11, 12, 19, 18, 0]
0.3979952321
0.6271598102
0.7494913893999999
0.7883679295999999
0.7883679295999999
maxIjogbon 0.266903016
cont_distance 0.669677499
C2 0.7883679295999999
sum 0.11777908857142855
[17, 0, 6, 13, 4]
0.24746530680000003
0.3417497392
0.4627858658999999
0.5303186039999999
0.5303186039999999
maxIjogbon 0.423034124
cont_distance 0.382952186
C2 0.5303186039999999
sum 0.12514932633333334
[17, 18, 15, 12, 19]
0.3390669605
0.602706127
0.8054775265000002
0.8967252188000001
0.8967252188000001
maxIjogbon 0.438764182
cont_distance 0.557618536
C2 0.8967252188000001
sum 0.14588798633333333
[1, 16, 17, 13, 4]
0.24632492369999998
0.5450104778
0.6891336531
0.7566663911999999
0.7566663911999999
maxIjogbon 0.446606

maxIjogbon 0.268416266
cont_distance 0.607472823
C2 0.9124482819999999
sum 0.10570119404761906
[9, 17, 13, 19, 16]
0.38194287709999997
0.6371856321999999
0.8118864454999999
0.9178998307999999
0.9178998307999999
maxIjogbon 0.34343714
cont_distance 0.642858658
C2 0.9178998307999999
sum 0.12759980314285715
[1, 3, 16, 5, 6]
0.20204492690000003
0.34240842380000003
0.5252632797000001
0.579546404
0.579546404
maxIjogbon 0.488235886
cont_distance 0.642858658
C2 0.579546404
sum 0.15161195557142856
[12, 11, 19, 1, 8]
0.3525190375
0.6408928836000001
0.7700909995
0.8158340764
0.8158340764
maxIjogbon 0.421022636
cont_distance 0.669677499
C2 0.8158340764
sum 0.15533290085714285
[2, 16, 3, 15, 8]
0.21427769219999998
0.4968522804
0.5938341067999999
0.676286262
0.676286262
maxIjogbon 0.44158265
cont_distance 0.642858658
C2 0.676286262
sum 0.15321809876190476
[18, 2, 10, 3, 14]
0.2877461269
0.45012351079999996
0.6280420503
0.6844660252000001
0.6844660252000001
maxIjogbon 0.44158265
cont_distance 0.607472

maxIjogbon 0.357032701
cont_distance 0.495201728
C2 0.7994606303999999
sum 0.12893537785714285
[8, 5, 9, 1, 12]
0.2536882243
0.4556781022
0.6258963299
0.694469278
0.694469278
maxIjogbon 0.369072145
cont_distance 0.508226398
C2 0.694469278
sum 0.14030543799999998
[5, 17, 0, 8, 15]
0.2308730938
0.42821090479999996
0.51066306
0.5931152151999999
0.5931152151999999
maxIjogbon 0.495201728
cont_distance 0.557618536
C2 0.5931152151999999
sum 0.13896565966666669
[19, 2, 4, 18, 9]
0.30486699100000003
0.4729613217999999
0.6290385499999999
0.7187377299999999
0.7187377299999999
maxIjogbon 0.268416266
cont_distance 0.508226398
C2 0.7187377299999999
sum 0.11578980652380953
[8, 13, 9, 6, 3]
0.2259440901
0.4138144018
0.5494762113
0.5834927411999999
0.5834927411999999
maxIjogbon 0.420310776
cont_distance 0.508226398
C2 0.5834927411999999
sum 0.14915909214285714
[14, 15, 3, 4, 19]
0.33457978999999993
0.58404673
0.6734929447999999
0.7484094883999999
0.7484094883999999
maxIjogbon 0.428657641
cont_distance 

...

maxIjogbon 0.428657641
cont_distance 0.642858658
C2 0.7733950931999999
sum 0.11960519433333333
[11, 0, 3, 2, 4]
0.3233935682
0.37891613679999997
0.441703541
0.48996127399999995
0.48996127399999995
maxIjogbon 0.44158265
cont_distance 0.669677499
C2 0.48996127399999995
sum 0.15385518866666667
[10, 5, 7, 2, 18]
0.3666753034
0.5454243375999999
0.6837885768999998
0.7377338259999998
0.7377338259999998
maxIjogbon 0.44158265
cont_distance 0.607472823
C2 0.7377338259999998
sum 0.14171150209523808
[17, 10, 13, 6, 3]
0.2822884006
0.5328904914000001
0.6355944493000001
0.6696109792
0.6696109792
maxIjogbon 0.365543138
cont_distance 0.607472823
C2 0.6696109792
sum 0.10274974585714285
[1, 4, 0, 16, 17]
0.2119812097
0.4141293664
0.5167104508
0.6192915352
0.6192915352
maxIjogbon 0.495201728
cont_distance 0.642858658
C2 0.6192915352
sum 0.14684151133333334
[17, 8, 4, 2, 0]
0.22812890900000005
0.3564575468000001
0.43790430390000007
0.4529730128000001
0.4529730128000001
maxIjogbon 0.44158265
cont_distance 

[13, 1, 17, 3, 2]
0.2170563944
0.3348434833999999
0.43376746509999997
0.45610100959999994
0.45610100959999994
maxIjogbon 0.446606371
cont_distance 0.382952186
C2 0.45610100959999994
sum 0.12954935376190477
[14, 7, 9, 3, 12]
0.3464537005
0.580689834
0.739120122
0.7959051304
0.7959051304
maxIjogbon 0.295152355
cont_distance 0.508226398
C2 0.7959051304
sum 0.11406410095238093
[11, 2, 17, 12, 0]
0.3707550999
0.5037766180000001
0.6298872280000001
0.6794074008
0.6794074008
maxIjogbon 0.268416266
cont_distance 0.669677499
C2 0.6794074008
sum 0.11631804228571427
[13, 6, 11, 4, 1]
0.28333609959999995
0.48280073179999994
0.668978031
0.7212198303999999
0.7212198303999999
maxIjogbon 0.310969875
cont_distance 0.669677499
C2 0.7212198303999999
sum 0.12024354614285714
[12, 1, 17, 14, 3]
0.31185266
0.4637301794
0.5967445915
0.6531685664
0.6531685664
maxIjogbon 0.428657641
cont_distance 0.495201728
C2 0.6531685664
sum 0.11785838695238096
[13, 14, 11, 17, 10]
0.3525442461
0.6660319147999999
0.8990099155

maxIjogbon 0.488235886
cont_distance 0.557618536
C2 0.7343610164000001
sum 0.12634589557142858
[3, 16, 11, 4, 7]
0.23571147729999997
0.5709353437999999
0.7802693627999999
0.8556678819999999
0.8556678819999999
maxIjogbon 0.488235886
cont_distance 0.669677499
C2 0.8556678819999999
sum 0.12820439876190476
[14, 8, 2, 10, 12]
0.3486630228
0.5540700916000001
0.6944749645000001
0.8047424196000001
0.8047424196000001
maxIjogbon 0.365543138
cont_distance 0.607472823
C2 0.8047424196000001
sum 0.11821654809523809
[13, 10, 8, 16, 1]
0.308151081
0.6004218706
0.7371411149
0.8204797559999999
0.8204797559999999
maxIjogbon 0.365543138
cont_distance 0.642858658
C2 0.8204797559999999
sum 0.13104077395238095
[7, 16, 6, 8, 1]
0.3056186174
0.5709709860000001
0.6702174515
0.7159605284
0.7159605284
maxIjogbon 0.539145578
cont_distance 0.642858658
C2 0.7159605284
sum 0.16485887266666666
[8, 11, 12, 16, 2]
0.30260370379999996
0.6323817009999999
0.8107766212999999
0.8901311959999999
0.8901311959999999
maxIjogbon 

maxIjogbon 0.488235886
cont_distance 0.508226398
C2 0.47523171799999997
sum 0.15479598995238095
[1, 11, 5, 3, 13]
0.2123188307
0.48236206
0.5790334696
0.6206420192000001
0.6206420192000001
maxIjogbon 0.369072145
cont_distance 0.669677499
C2 0.6206420192000001
sum 0.1305982991904762
[12, 14, 9, 0, 19]
0.3397899898
0.5798175670000001
0.7231903661000001
0.7649178856000001
0.7649178856000001
maxIjogbon 0.263721654
cont_distance 0.508226398
C2 0.7649178856000001
sum 0.1137646996190476
[9, 0, 8, 11, 6]
0.323700305
0.4441100508000001
0.5912100982000001
0.6849295424000001
0.6849295424000001
maxIjogbon 0.495201728
cont_distance 0.669677499
C2 0.6849295424000001
sum 0.1971125881428571
[4, 10, 9, 2, 15]
0.315156581
0.6190516302
0.7915274723000001
0.8623580348000001
0.8623580348000001
maxIjogbon 0.44158265
cont_distance 0.607472823
C2 0.8623580348000001
sum 0.12583300247619048
[15, 19, 16, 0, 3]
0.3363256353
0.5330588951999999
0.6688954623999999
0.6761602979999999
0.6761602979999999
maxIjogbon 0.4

maxIjogbon 0.44158265
cont_distance 0.642858658
C2 0.7573526396
sum 0.14203901014285714
[11, 6, 16, 0, 10]
0.41965584199999995
0.624944073
0.8142630868999999
0.8750103692
0.8750103692
maxIjogbon 0.495201728
cont_distance 0.669677499
C2 0.8750103692
sum 0.15429096995238095
[18, 4, 17, 10, 14]
0.3368968251
0.5846655376000001
0.7711623964000001
0.8810688179999999
0.8810688179999999
maxIjogbon 0.509327753
cont_distance 0.607472823
C2 0.8810688179999999
sum 0.17846106890476188
[12, 9, 19, 3, 1]
0.3169484614
0.5374615112
0.6472341610999999
0.6735517719999999
0.6735517719999999
maxIjogbon 0.421022636
cont_distance 0.508226398
C2 0.6735517719999999
sum 0.1506178873333333
[16, 17, 9, 0, 6]
0.37301301589999997
0.5654730058
0.6938699797
0.720621674
0.720621674
maxIjogbon 0.495201728
cont_distance 0.642858658
C2 0.720621674
sum 0.1279230800952381
[7, 11, 15, 17, 2]
0.3449315114
0.6549605422000001
0.8198481769000001
0.8732121044000001
0.8732121044000001
maxIjogbon 0.44158265
cont_distance 0.6696774

In [31]:
print (len(Particles))

455


In [32]:
# MOPSO main loop
for it in tqdm(range(maxIt)):
    for i in range(nPop):
        leader = SelectLeader(Repos,beta)
        print ("leader", leader)
        # update velocity
        print ("Particles[i].best_position", Particles[i].best_position, "type", 
               type(Particles[i].best_position))
        #leader.position = np.array(leader)
        leaderpos_1 = leader.position

        
        
        positions = zip(Particles[i].best_position, Particles[i].position)
        position_diff = [x - y for (x, y) in positions]
        
        leaders = zip(leaderpos_1, Particles[i].position)
        leader_pos = [x - y for (x, y) in leaders]
        
        Particles[i].velocity = w*Particles[i].velocity  \
            + c1*np.random.rand(1,nVar)[0]*(position_diff) \
            + c2*np.random.rand(1,nVar)[0]*(leader_pos)
        
       
              
        # update position
        Particles[i].position = Particles[i].position + Particles[i].velocity 

        # evaluation
        tp = fixer(Particles[i].position)
        Particles[i].position = tp
        Particles[i].cost = MOCPParticle(Particles[i].position)
        

        if Dominates(Particles[i].cost,Particles[i].best_cost):
            Particles[i].best_position = Particles[i].position
            Particles[i].best_cost = Particles[i].cost
        else:
            if np.random.rand() > 0.5:
                Particles[i].best_position = Particles[i].position
                Particles[i].best_cost = Particles[i].cost
      
   
    Repos = Repos + Particles
    Repos = DetermineDomination(Repos)
    Repos = [item for item in Repos if item.IsDominated == False ]

    grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)
    for r in range(len(Repos)):
        Repos[r] = FindGridIndex(Repos[r],grid)

    # check if repository is full
    if len(Repos) > nRep :
        extra = len(Repos) - nRep
        for e in range(extra):
            Repos = deleteOneRepositoryMember(Repos,gamma)

    ########## show figure ########## 
    plt.clf()
    particlesCost = np.reshape( [item.cost for item in Particles ],newshape=(nPop,5))
    repositoryCost = [item.cost for item in Repos]
    repositoryCost = np.reshape( repositoryCost, newshape=(len(repositoryCost),5))
    plt.plot(particlesCost[:,0], particlesCost[:,1], 'o' ,mfc='none')
    plt.plot(repositoryCost[:,0], repositoryCost[:,1], 'r*')
    
    plt.draw()
    plt.pause(0.00000000001)

    w=w*wdamping
    
    # print(repositoryCost)
    # print("ok")
    # print(particlesCost)
    ########## show figure ##########
plt.show()

  0%|          | 0/5 [00:00<?, ?it/s]

leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 7 11  1 14  3] type <class 'numpy.ndarray'>
[11, 8, 2, 9, 17]
0.3987478685
0.5830053406
0.7022606168000001
0.7913784752000002
0.7913784752000002
maxIjogbon 0.44158265
cont_distance 0.669677499
C2 0.7913784752000002
sum 0.1338842151904762
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [14  1  3 11 13] type <class 'numpy.ndarray'>
[14, 1, 3, 11, 13]
0.324265632
0.4900002574
0.6058413925
0.7071528563999999
0.7071528563999999
maxIjogbon 0.299491969
cont_distance 0.669677499
C2 0.7071528563999999
sum 0.09777109776190475
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [12  0  7 19 18] type <class 'numpy.ndarray'>
[15, 0, 6, 7, 10]
0.3527558861
0.48246435779999997
0.6389245238000001
0.7418813012000001
0.7418813012000001
maxIjogbon 0.495201728
cont_distance 0.607472823
C2 0.7418813012000001
sum 0.15762675119047617
leader <__main__.Particle 

maxIjogbon 0.495201728
cont_distance 0.642858658
C2 0.6367523279999999
sum 0.1562622468095238
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [12 16  7 10 17] type <class 'numpy.ndarray'>
[15, 2, 1, 11, 16]
0.38842251429999997
0.583935032
0.7532941983
0.8845478139999999
0.8845478139999999
maxIjogbon 0.44158265
cont_distance 0.669677499
C2 0.8845478139999999
sum 0.13665375090476192
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 7 13 15  4 17] type <class 'numpy.ndarray'>
[10, 0, 1, 6, 13]
0.3231373128
0.4032854964
0.5024864553
0.5635818636000001
0.5635818636000001
maxIjogbon 0.365543138
cont_distance 0.607472823
C2 0.5635818636000001
sum 0.12388123642857142
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 0 13 12  3  7] type <class 'numpy.ndarray'>
[7, 0, 2, 14, 13]
0.2674095426
0.3659811048
0.4796213759
0.5631242291999999
0.5631242291999999
maxIjogbon 0.357032701
cont_distance 0

maxIjogbon 0.44158265
cont_distance 0.669677499
C2 0.794256496
sum 0.12321708533333332
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 6 10 18  5 16] type <class 'numpy.ndarray'>
[19, 6, 0, 10, 13]
0.2887527686
0.4640988478
0.5591898440999999
0.6542808404
0.6542808404
maxIjogbon 0.365543138
cont_distance 0.607472823
C2 0.6542808404
sum 0.12294838228571427
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [19  3  9  4 17] type <class 'numpy.ndarray'>
[15, 2, 3, 8, 12]
0.32159143329999995
0.45027286999999994
0.5410130155999999
0.61722349
0.61722349
maxIjogbon 0.403059166
cont_distance 0.557618536
C2 0.61722349
sum 0.1248149940952381
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 6 19 15  5  8] type <class 'numpy.ndarray'>
[9, 0, 1, 10, 12]
0.3326107896
0.4619310199999999
0.6103040256999999
0.7205714807999999
0.7205714807999999
maxIjogbon 0.285290274
cont_distance 0.607472823
C2 0.7

maxIjogbon 0.264922331
cont_distance 0.669677499
C2 0.8483756283999998
sum 0.10569894785714286
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [18 12  9 17 14] type <class 'numpy.ndarray'>
[10, 0, 8, 13, 12]
0.35354331759999996
0.464097506
0.601341996
0.6852058828
0.6852058828
maxIjogbon 0.365543138
cont_distance 0.607472823
C2 0.6852058828
sum 0.12439510252380954
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [18 11 14  8  2] type <class 'numpy.ndarray'>
[14, 0, 4, 8, 9]
0.30733852269999995
0.41804048819999995
0.5619314777999999
0.6394444192
0.6394444192
maxIjogbon 0.495201728
cont_distance 0.508226398
C2 0.6394444192
sum 0.12423758038095238
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [16 15 14 19  1] type <class 'numpy.ndarray'>
[15, 6, 0, 13, 16]
0.3484286885
0.5273133512
0.625942931
0.7245725108000001
0.7245725108000001
maxIjogbon 0.423034124
cont_distance 0.642858658
C2 0.

...

maxIjogbon 0.480250491
cont_distance 0.669677499
C2 0.6620470752
sum 0.17222936209523806
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [14  7 17  8 10] type <class 'numpy.ndarray'>
[14, 5, 0, 13, 15]
0.3142735548
0.48697341239999997
0.5770789799999999
0.6671845476
0.6671845476
maxIjogbon 0.428657641
cont_distance 0.557618536
C2 0.6671845476
sum 0.12101391776190475
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 7  3 10 16 17] type <class 'numpy.ndarray'>
[14, 0, 1, 10, 15]
0.3321984684
0.46776037959999994
0.6223750660999999
0.7388842019999999
0.7388842019999999
maxIjogbon 0.495201728
cont_distance 0.607472823
C2 0.7388842019999999
sum 0.16101040804761904
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 4  6 12 13 19] type <class 'numpy.ndarray'>
[5, 0, 6, 10, 16]
0.2619105624
0.4136954048
0.5922319415
0.7172650896
0.7172650896
maxIjogbon 0.495201728
cont_distance 0.642858658
C2

[12, 13, 10, 18, 14]
0.381207367
0.6330214708
0.8425517149000001
0.9305873944000002
0.9305873944000002
maxIjogbon 0.379449102
cont_distance 0.607472823
C2 0.9305873944000002
sum 0.1467465444285714
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 7 16 12  2  4] type <class 'numpy.ndarray'>
[14, 0, 1, 11, 12]
0.33217725519999997
0.4677179531999999
0.6223114264999999
0.7387993492
0.7387993492
maxIjogbon 0.285290274
cont_distance 0.669677499
C2 0.7387993492
sum 0.10489781342857142
leader <__main__.Particle object at 0x00000221028070A0>
Particles[i].best_position [ 2  9 14 19  6] type <class 'numpy.ndarray'>
[15, 2, 7, 11, 16]
0.41157923409999997
0.6302484715999999
0.8459210774999999
0.9771746931999999
0.9771746931999999
maxIjogbon 0.44158265
cont_distance 0.669677499
C2 0.9771746931999999
sum 0.13678491714285715


  0%|          | 0/5 [00:50<?, ?it/s]


ValueError: cannot reshape array of size 1820 into shape (455,5)

<Figure size 640x480 with 0 Axes>

In [ ]:
 repositoryCost

In [ ]:
particlesCost

In [ ]:
#This is the initial population
Temp_Particles

In [ ]:
inds = Temp_Particles
indsDf = pd. DataFrame(inds)
indsDf.sample(2)

In [ ]:
indsDf

In [ ]:
Temp_Position

In [ ]:
Particles[0].position

In [ ]:
tp

In [ ]:
Particles[i].cost

In [ ]:
leader_pos

# ABOVE CODE ORIGINAL TEMPLATE WITHOUT EDITING

In [ ]:
import numpy as np
from numpy import matlib
import matplotlib.pyplot as plt
import random as random
import math

In [ ]:
def deleteOneRepositoryMember(rep , gamma):
    gridindices = [item.gridIndex for item in rep]
    OCells = np.unique(gridindices) # ocupied cells
    N = np.zeros(len(OCells))
    for k in range(len(OCells)):
        N[k] = gridindices.count(OCells[k])
    # selection probablity
    p = [math.exp(gamma*item) for item in N]
    p = np.array(p)/sum(p)

    # select cell index
    sci = roulettewheelSelection(p)
    SelectedCell = OCells[sci]

    #selected Cell members
    selectedCellmembers = [item for item in gridindices if item == SelectedCell]

    selectedmemberindex = np.random.randint(0,len(selectedCellmembers))
    #selectedmember = selectedCellmembers[selectedmemberindex]

    # delete memeber
    #rep[selectedmemberindex] = []
    rep = np.delete(rep, selectedmemberindex)

    return rep.tolist()

In [ ]:
def SelectLeader(rep , beta):
    gridindices = [item.gridIndex for item in rep]
    OCells = np.unique(gridindices) # ocupied cells
    N = np.zeros(len(OCells))
    for k in range(len(OCells)):
        N[k] = gridindices.count(OCells[k])
    # selection probablity
    p = [math.exp(-beta*item) for item in N]
    p = np.array(p)/sum(p)

    # select cell index
    sci = roulettewheelSelection(p)
    SelectedCell = OCells[sci]

    #selected Cell members
    selectedCellmembers = [item for item in gridindices if item == SelectedCell]

    selectedmemberindex = np.random.randint(0,len(selectedCellmembers))
    # selectedmember = selectedCellmembers[selectedmemberindex]

    return rep[selectedmemberindex]

In [ ]:
def roulettewheelSelection(p):
    r = random.random()
    cumsum = np.cumsum(p)
    y = (cumsum<r)
    x= [i for i in y if i==True]
    return len(x)

In [ ]:
def FindGridIndex(particle, grid):
    nObj = len(particle.cost)
    NGrid = len(grid[0].LowerBounds)
    
    particle.gridSubIndex = np.zeros((1,nObj))[0]
    for j in range(nObj):  
        index_in_Dim = len( [item for item in grid[j].UpperBounds if particle.cost[j]>item]) 
        particle.gridSubIndex[j] = index_in_Dim

    particle.gridIndex = particle.gridSubIndex[0]

    for j in range(1,nObj):
        particle.gridIndex = particle.gridIndex 
        particle.gridIndex = NGrid*particle.gridIndex
        particle.gridIndex = particle.gridIndex + particle.gridSubIndex[j]

    return particle

In [ ]:
def CreateGrid(pop,nGrid,alpha,nobj):
    costs = [item.cost for item in pop]
    Cmin = np.min(costs,axis=0)
    Cmax = np.max(costs,axis=0)
    deltaC = Cmax - Cmin
    Cmin =  Cmin - alpha*deltaC
    Cmax = Cmax + alpha*deltaC
   
    grid = [GridDim() for p in range(nobj)]
    for i in range(nobj):
       dimValues = np.linspace(Cmin[i],Cmax[i],nGrid+1).tolist()
       grid[i].LowerBounds = [-float('inf')] + dimValues
       grid[i].UpperBounds = dimValues  + [float('inf')]
    return grid

In [ ]:
def Dominates(x,y):
    x=np.array(x)
    y=np.array(y)
    x_dominate_y = all(x<=y) and any(x<y)
    return x_dominate_y

def DetermineDomination(pop):
    pop_len= len(pop)
    for i in range(pop_len):
         pop[i].IsDominated = False 

    for i in range(pop_len-1):
        for j in range(i+1,pop_len):
            if Dominates(pop[i].cost,pop[j].cost):
                pop[j].IsDominated = True
            if Dominates(pop[j].cost,pop[i].cost):
                pop[i].IsDominated = True

    return pop

In [ ]:
# problem definition
def MOP2(x):
    #print (type(x))
    print (x)
    x = np.array(x)
    n= len(x)
    z1 = 1 - math.exp(-sum((x-1/math.sqrt(n))**2))
    z2 = 1 - math.exp(-sum((x+1/math.sqrt(n))**2))
    return [z1,z2]

costfunction = lambda x: MOP2(x)

In [ ]:
#costfunction


In [ ]:
nVar = 5 # number of decision vars
varMin = -4
varMax = 4
maxIt = 100
nPop = 200    # population size
nRep = 50  # size of repository
w = 0.5 # inertia wieght
c1 = 2 # personal learning coefficient
c2 = 2 # global learning coefficient
wdamping = 0.99


In [ ]:
# ################ constriction coefficients
# phi1 = 2.05
# phi2 = 2.05
# phi = phi1+phi2
# chi = 2/(phi - 2 + np.sqrt(phi**2 - 4*phi))
# w = chi # inertia wieght
# c1 = chi*phi1 # personal learning coefficient
# c2 = chi*phi2 # global learning coefficient
# wdamping = 1
# #################

beta = 1 # leader selection pressure
gamma = 1 # deletion selection pressure
NoGrid = 5
alpha=0.1 # nerkhe tavarrom grid

# initialization
class Particle:
    position = []
    cost = []
    velocity = []
    best_position = []
    best_cost = []
    IsDominated = []
    gridIndex = []
    gridSubIndex = []

# for each objective a grid items is division of values of objective cost
class GridDim: 
    LowerBounds = []
    UpperBounds = []

#Particles = np.matlib.repmat(Particle,nPop,1)
Particles = [Particle() for p in range(nPop)]
print (Particles)
for i in range(nPop):
    Particles[i].position = np.random.uniform(varMin,varMax,nVar)
    Particles[i].velocity = np.zeros(nVar)
    Particles[i].cost = costfunction(Particles[i].position)
    # update best personal Best
    Particles[i].best_position = Particles[i].position
    Particles[i].best_cost = Particles[i].cost
    Particles[i].IsDominated = False 

Particles = DetermineDomination(Particles)

Repos = [item for item in Particles if item.IsDominated == False ]
#print (Repos[0].cost)
nObj =len( Repos[0].cost)
grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)

for r in range(len(Repos)):
    Repos[r] = FindGridIndex(Repos[0],grid)
    #print("r is", r)
    #print ("repos r is", r)


In [ ]:
# MOPSO main loop
for it in range(maxIt):
    for i in range(nPop):
        leader = SelectLeader(Repos,beta)
        # update velocity
        Particles[i].velocity = w*Particles[i].velocity  \
            + c1*np.random.rand(1,nVar)[0]*(Particles[i].best_position - Particles[i].position) \
            + c2*np.random.rand(1,nVar)[0]*(leader.position - Particles[i].position)
              
        # update position
        Particles[i].position = Particles[i].position + Particles[i].velocity 

        # evaluation
        Particles[i].cost = costfunction(Particles[i].position)

        if Dominates(Particles[i].cost,Particles[i].best_cost):
            Particles[i].best_position = Particles[i].position
            Particles[i].best_cost = Particles[i].cost
        else:
            if np.random.rand() > 0.5:
                Particles[i].best_position = Particles[i].position
                Particles[i].best_cost = Particles[i].cost
      

    Repos = Repos + Particles
    Repos = DetermineDomination(Repos)
    Repos = [item for item in Repos if item.IsDominated == False ]

    grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)
    for r in range(len(Repos)):
        Repos[r] = FindGridIndex(Repos[r],grid)

    # check if repository is full
    if len(Repos) > nRep :
        extra = len(Repos) - nRep
        for e in range(extra):
            Repos = deleteOneRepositoryMember(Repos,gamma)

    ########## show figure ########## 
    plt.clf()
    particlesCost = np.reshape( [item.cost for item in Particles ],newshape=(nPop,2))
    repositoryCost = [item.cost for item in Repos]
    repositoryCost = np.reshape( repositoryCost, newshape=(len(repositoryCost),2))
    plt.plot(particlesCost[:,0], particlesCost[:,1], 'o' ,mfc='none')
    plt.plot(repositoryCost[:,0], repositoryCost[:,1], 'r*')
    
    plt.draw()
    plt.pause(0.00000000001)

    w=w*wdamping
    
    # print(repositoryCost)
    # print("ok")
    # print(particlesCost)
    ########## show figure ##########
plt.show()

# MOPSO TEMPLATE CODE3 FOR CONTROLLER PLACEMENT

In [ ]:
# FUNCTION TO CHECK NON DOMINATION OF PARTICLES IN POPULATION AND ARCHIVE

import random
from math import *
import numpy
import copy



def check_nondom(i,archive_temp,no_of_obj_func,ARCHIVE_FIT,PARTICLE_FIT,COUNT,ARCHIVE_VAL):
    h=0
    while(h<=int(archive_temp[0])):
        summ = 0;
        if((ARCHIVE_FIT[h][0]<PARTICLE_FIT[i][0]) and (ARCHIVE_FIT[h][1]<PARTICLE_FIT[i][1]) and (ARCHIVE_FIT[h][2]>PARTICLE_FIT[i][2])):
            summ+=no_of_obj_func   # if particle in archive dominates
            return(0)
        elif((ARCHIVE_FIT[h][0]>=PARTICLE_FIT[i][0]) and (ARCHIVE_FIT[h][1]>=PARTICLE_FIT[i][1]) and (ARCHIVE_FIT[h][2]<=PARTICLE_FIT[i][2])):
            summ=0     # if particle in archive is dominated , delete it
            for j in range(COUNT):
                ARCHIVE_VAL[h][j]=ARCHIVE_VAL[int(archive_temp[0])-1][j]
            for j in range(no_of_obj_func):
                ARCHIVE_FIT[h][j]=ARCHIVE_FIT[int(archive_temp[0])-1][j]
            archive_temp[0]-=1
        else:
            h+=1
            
    return(1)

In [ ]:
# COMPUTE CROWDING DISTANCES OF PARTICLES IN ARCHIVE

import nbimporter
import random
from math import *
import copy
import numpy
import FITNESS_SORT
import DISTANCE_COMPUTATION
import CROWD_SORT
reload(CROWD_SORT)
reload(DISTANCE_COMPUTATION)
reload(FITNESS_SORT)


def crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL):
    CrowdList=[float(0.0) for i in range(no_of_nondom_sol)]   #initialise crowding distance values
    # objective function 1
    begin=0
    # sort fitness values
    no_of_nondom_sol=FITNESS_SORT.sort_fitness(0,begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,1)
    # compute crowding distance
    no_of_nondom_sol=DISTANCE_COMPUTATION.compute_distance(0,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,1)
    # objective function 2
    begin=0
    # sort fitness values
    no_of_nondom_sol=FITNESS_SORT.sort_fitness(1,begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,1)
    # compute crowding distance
    no_of_nondom_sol=DISTANCE_COMPUTATION.compute_distance(1,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,1)
    # objective function 3
    begin=0
    # sort fitness values
    no_of_nondom_sol=FITNESS_SORT.sort_fitness(2,begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,2)
    # compute crowding distance
    no_of_nondom_sol=DISTANCE_COMPUTATION.compute_distance(2,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,2)
    
    begin=0
    # sort crowding distance values
    no_of_nondom_sol=CROWD_SORT.sort_crowd(begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList)
    return(no_of_nondom_sol)

In [ ]:
import random
from math import *
import copy
import numpy
import nbimporter
import CROWD_SORT
reload(CROWD_SORT)


def sort_crowd(begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList):
    l=begin+1
    r=lastpart
    turn=CrowdList[begin]
    flagF=[[] for i in range(no_of_obj_func)]
    flagP=[[] for i in range(COUNT)]
    while(l<r):
        if(CrowdList[l]<=turn):
            l+=1
        else:
            r-=1
            # swap the crowding distance values
            temp=CrowdList[l]
            CrowdList[l]=CrowdList[r]
            CrowdList[r]=temp
            # swap fitness values of two particles in the archive
            for k in range(no_of_obj_func):
                flagF[k]=ARCHIVE_FIT[l][k]
                ARCHIVE_FIT[l][k]=ARCHIVE_FIT[r][k]
                ARCHIVE_FIT[r][k]=flagF[k]
            # swap position values of two particles in the archive    
            for k in range(COUNT):
                flagP[k]=ARCHIVE_VAL[l][k]
                ARCHIVE_VAL[l][k]=ARCHIVE_VAL[r][k]
                ARCHIVE_VAL[r][k]=flagP[k]
           
    return(lastpart)
#     #swap fitness values of two particles in the archive
#     for k in range(no_of_obj_func):
#         flagF[k]=ARCHIVE_FIT[begin][k]
#         ARCHIVE_FIT[begin][k]=ARCHIVE_FIT[l][k]
#         ARCHIVE_FIT[l][k]=flagF[k]
#      # swap position values of two particles in the archive 
#     for k in range(COUNT):
#         flagP[k]=ARCHIVE_VAL[begin][k]
#         ARCHIVE_VAL[begin][k]=ARCHIVE_VAL[l][k]
#         ARCHIVE_VAL[l][k]=flagP[k]
#     # swap their crowding distance values
#     temp = CrowdList[begin];
#     CrowdList[begin] = CrowdList[l];
#     CrowdList[l] = temp;
    
#     if((l-begin)>1):
#         lastpart=CROWD_SORT.sort_crowd(begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList)
#     if((lastpart-r)>1):
#         lastpart=CROWD_SORT.sort_crowd(begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList)

In [ ]:
# FUNCTION TO DELETE PARTICLE IN ARCHIVE

import random
from math import *
import copy
import numpy


def delete_particle(k,no_of_obj_func,no_of_nondom_sol,ARCHIVE_FIT):
    # if infeasible particle is the last in archive or only one particle in archive
    if((no_of_nondom_sol== 1) | (k == no_of_nondom_sol-1)):
        no_of_nondom_sol-=1
    # move last particle in archive in place of infeasible particle
    else:
        for j in range(no_of_obj_func):
            ARCHIVE_FIT[k][j]=ARCHIVE_FIT[no_of_nondom_sol-1][j]
        no_of_nondom_sol-= 1;
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO COMPUTE CROWDING DISTANCE NON DOMINATED PARTICLES

import random
from math import *
import copy

def compute_distance(f,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,am):
    maximum=1
    minimum=1
    if(am==1):
        for i in range(1,no_of_nondom_sol-1):
            print type(ARCHIVE_FIT[i][f][0])
            CrowdList[i]=CrowdList[i]+((ARCHIVE_FIT[i+1][f])-(ARCHIVE_FIT[i-1][f]))
            if(CrowdList[minimum]>CrowdList[i]):
                minimum=i
        CrowdList[0]=CrowdList[0]+CrowdList[minimum]
        CrowdList[no_of_nondom_sol-1]=CrowdList[no_of_nondom_sol-1]+CrowdList[minimum]
        
    elif(am==2):
        for i in range(1,no_of_nondom_sol-1):
            print type(ARCHIVE_FIT[i][f])
            CrowdList[i]=CrowdList[i]+((ARCHIVE_FIT[i+1][f])-(ARCHIVE_FIT[i-1][f]))
            if(CrowdList[maximum]<CrowdList[i]):
                maximum=i
        CrowdList[0]=CrowdList[0]+CrowdList[maximum]
        CrowdList[no_of_nondom_sol-1]=CrowdList[no_of_nondom_sol-1]+CrowdList[maximum]
    
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO SORT FITNESS VALUES OF PARTICLES IN ARCHIVE

import random
from math import *
import copy
import numpy
import nbimporter
import FITNESS_SORT
reload(FITNESS_SORT)


def sort_fitness(f,begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,am):
    l=begin+1
    r=lastpart
    turn=ARCHIVE_FIT[begin][f]
    flagF=[[] for i in range(no_of_obj_func)]
    flagP=[[] for i in range(COUNT)]
    while(l<r):
        if(am==1):
            if(ARCHIVE_FIT[l][f]>=turn):
                l+=1
            else:
                r-=1
                # swap fitness values of two particles in the archive
                for k in range(no_of_obj_func):
                    flagF[k]=ARCHIVE_FIT[l][k]
                    ARCHIVE_FIT[l][k]=ARCHIVE_FIT[r][k]
                    ARCHIVE_FIT[r][k]=flagF[k]
                #swap positions of two particles in the archive
                for k in range(COUNT):
                    flagP[k]=ARCHIVE_VAL[l][k]
                    ARCHIVE_VAL[l][k]=ARCHIVE_VAL[r][k]
                    ARCHIVE_VAL[r][k]=flagP[k]
                # swap their crowding distances
                temp=CrowdList[l]
                CrowdList[l]=CrowdList[r]
                CrowdList[r]=temp
                
        elif(am==2):
            if(ARCHIVE_FIT[l][f]<=turn):
                l+=1
            else:
                r-=1
                # swap fitness values of two particles in the archive
                for k in range(no_of_obj_func):
                    flagF[k]=ARCHIVE_FIT[l][k]
                    ARCHIVE_FIT[l][k]=ARCHIVE_FIT[r][k]
                    ARCHIVE_FIT[r][k]=flagF[k]
                #swap positions of two particles in the archive
                for k in range(COUNT):
                    flagP[k]=ARCHIVE_VAL[l][k]
                    ARCHIVE_VAL[l][k]=ARCHIVE_VAL[r][k]
                    ARCHIVE_VAL[r][k]=flagP[k]
                # swap their crowding distances
                temp=CrowdList[l]
                CrowdList[l]=CrowdList[r]
                CrowdList[r]=temp
                
            
                
#     l-=1
#     # swap fitness values of two particles in the archive
#     for k in range(no_of_obj_func):
#         flagF[k]=ARCHIVE_FIT[begin][k]
#         ARCHIVE_FIT[begin][k]=ARCHIVE_FIT[l][k]
#         ARCHIVE_FIT[l][k]=flagF[k]
#     # swap positions of two particles in the archive
#     for k in range(COUNT):
#         flagP[k]=ARCHIVE_VAL[begin][k]
#         ARCHIVE_VAL[begin][k]=ARCHIVE_VAL[l][k]
#         ARCHIVE_VAL[l][k]=flagP[k]
#     # swap their crowding distances
#     temp = CrowdList[begin];
#     CrowdList[begin] = CrowdList[l];
#     CrowdList[l] = temp;
    
#     if(l-begin>1):
#         lastpart=FITNESS_SORT.sort_fitness(f,begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,am)
#     if((lastpart-r)>1):
#         lastpart=FITNESS_SORT.sort_fitness(f,begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,am)
    return(lastpart)

In [ ]:
# FUNCTION TO CALCULATE THE INTERACTION SCORE OF EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
from itertools import combinations

def interaction_score(POPULATION_SIZE,no_of_pathways,SUM):
    DEG_OF_RELEVANCE=[random.uniform(0,1) for i in range(no_of_pathways)] # set degree of relevance of each pathway
    WEIGHTED_SUM=0.0
    WEIGHTED_VAL=0.0
    for i in range(no_of_pathways):
        SUM[i]=SUM[i]*DEG_OF_RELEVANCE[i]
        WEIGHTED_SUM+=SUM[i]
        WEIGHTED_VAL+=DEG_OF_RELEVANCE[i]
        
    return(WEIGHTED_SUM/WEIGHTED_VAL)
        
    
        
#if __name__=='__main__':
def Interaction_Score(POPULATION_SIZE):
    no_of_pathways=0
    NO_OF_GENES = []
    arr1=[]
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
                
    #print NO_OF_GENES
    arr2=[]
    arr3=[]
    UNI_GENES=[[[] for j in range(NO_OF_GENES[i])] for i in range(no_of_pathways)]
    i=0
    j=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/GENES_UNIPROTKB_ID.txt") as am:
        for line in am:
            arr2=line.split("\t\t")
            arr3=arr2[1].split("\n")
            UNI_GENES[i][j]=arr3[0]
            if(j<(NO_OF_GENES[i]-1)):
                i=i
                j=j+1
            elif(j==NO_OF_GENES[i]-1):
                i+=1
                j=0
                
    UNI_GENES_PAIR=[[] for i in range(no_of_pathways)]
    for i in range(no_of_pathways):
        UNI_GENES_PAIR[i]=list(combinations(UNI_GENES[i],2))
    NO_OF_PAIR=[(len(UNI_GENES_PAIR[i])) for i in range(no_of_pathways)]
    for i in range(no_of_pathways):
        for j in range(NO_OF_PAIR[i]):
            UNI_GENES_PAIR[i][j]=list(UNI_GENES_PAIR[i][j])
    #print (UNI_GENES_PAIR[0][0][1])
    
    count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/H_sapiens_interaction_output.txt") as am:
        for line in am:
            count+=1
    #print count
    
    ALL_GENES=[[] for i in range(count)]
    i=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/H_sapiens_interaction_output.txt") as am:
        for line in am:
            ALL_GENES[i]=line.split("\t")
            ALL_GENES[i][4]=ALL_GENES[i][4][:-2]
            i+=1
    #print ALL_GENES[10]
    
    SUM=[float(0.0) for i in range(no_of_pathways)]
    for i in range(no_of_pathways):
        for j in range(NO_OF_PAIR[i]):
            for k in range(count):
                if((UNI_GENES_PAIR[i][j][0]==ALL_GENES[k][0] and UNI_GENES_PAIR[i][j][1]==ALL_GENES[k][1]) | (UNI_GENES_PAIR[i][j][0]==ALL_GENES[k][1] and UNI_GENES_PAIR[i][j][1]==ALL_GENES[k][0])):
                    #print UNI_GENES_PAIR[i][j][0],"\t",UNI_GENES_PAIR[i][j][1],"\t",ALL_GENES[k][0],"\t",ALL_GENES[k][1]
                    SUM[i]+=float(ALL_GENES[k][4])
#     POPULATION_SIZE=10
    
    INTERACTION_SCORE=[[] for i in range(POPULATION_SIZE)]
    
    for i in range(POPULATION_SIZE):
        Sum=copy.copy(SUM)
        INTERACTION_SCORE[i]=interaction_score(POPULATION_SIZE,no_of_pathways,SUM)
        SUM=copy.copy(Sum)
#     print INTERACTION_SCORE
    return(INTERACTION_SCORE)

In [ ]:
# FUNCTION TO CALCULATE FITNESS VALUES OF PARTICLES IN POPULATION

import nbimporter
import random
import WEIGHTED_T_SCORE
import WEIGHTED_Z_SCORE
import INTERACTION_SCORE
reload(WEIGHTED_T_SCORE)
reload(WEIGHTED_Z_SCORE)
reload(INTERACTION_SCORE)


def update_particle(PARTICLE_FIT,no_of_obj_function,POP_SIZE):
    T_SCORE=WEIGHTED_T_SCORE.Weighted_T_Score(POP_SIZE) 
    Z_SCORE=WEIGHTED_Z_SCORE.Weighted_Z_Score(POP_SIZE)
    I_SCORE=INTERACTION_SCORE.Interaction_Score(POP_SIZE)
    for i in range(int(POP_SIZE)):
        PARTICLE_FIT[i][0]=T_SCORE[i]
        PARTICLE_FIT[i][1]=Z_SCORE[i]
        PARTICLE_FIT[i][2]=I_SCORE[i]
        
    return(PARTICLE_FIT)

In [ ]:
# FUNCTION TO INSERT NEW NON DOMINATED PARTICLES FROM POPULATION TO ARCHIVE

import random
from math import *
import numpy
import copy
import nbimporter
import CHECK_NONDOMINATED_SOLUTION
import CROWDING
reload(CROWDING)
reload(CHECK_NONDOMINATED_SOLUTION)




def update_archive(SWARM_SIZE,no_of_nondom_sol,archive_size,COUNT,ARCHIVE_VAL,PARTICLE_VAL,ARCHIVE_FIT,PARTICLE_FIT,no_of_obj_func):
    i=0
    for k in range(SWARM_SIZE):
        archive_temp=[int(no_of_nondom_sol) for j in range(1)]
        # if particle in population is non dominated 
        if(CHECK_NONDOMINATED_SOLUTION.check_nondom(k,archive_temp,no_of_obj_func,ARCHIVE_FIT,PARTICLE_FIT,COUNT,ARCHIVE_VAL)==1):
            no_of_nondom_sol=int(archive_temp[0])
            if(no_of_nondom_sol < archive_size): # if archive memory is not full insert particle
                i=no_of_nondom_sol 
                for j in range(COUNT):
                    ARCHIVE_VAL[i][j]=PARTICLE_VAL[k][j]
                for j in range(no_of_obj_func):
                    ARCHIVE_FIT[i][j]=PARTICLE_FIT[k][j]
                no_of_nondom_sol+=1
            else: # if memory is full , select particle to replace
                # compute crowding distances of particles in archive
                no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
                bottom=(no_of_nondom_sol-1)*0.90
                i=random.randint(bottom,no_of_nondom_sol-1)
                # insert new particles in archive
                for j in range(COUNT):
                    ARCHIVE_VAL[i][j]=PARTICLE_VAL[k][j]
                for j in range(no_of_obj_func):
                    ARCHIVE_FIT[i][j]=PARTICLE_FIT[i][j]
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO COMPUTE NEW VELOCITY AND POSITION FOR INDIVIDUAL PARTICLE IN POPULATION

import random
from math import *
import numpy
import copy

def update_velocity_position(no_of_nondom_sol,SWARM_SIZE,COUNT,VELOCITY,pBestPosition,PARTICLE_VAL,ARCHIVE_VAL):
    flag=(no_of_nondom_sol-1)*0.10
    for i in range(SWARM_SIZE):
        gBest=random.randint(0,int(flag))
        # compute new velocities for particles
        for j in range(COUNT):
            VELOCITY[i][j]=0.79*VELOCITY[i][j]+2.0*random.random()*(pBestPosition[i][j]-PARTICLE_VAL[i][j])+2.0*random.random()*(ARCHIVE_VAL[gBest][j]-PARTICLE_VAL[i][j])
    # compute new positions for each particle in the population
    for i in range(SWARM_SIZE):
        for j in range(COUNT):
            PARTICLE_VAL[i][j]=PARTICLE_VAL[i][j]+VELOCITY[i][j]
            
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO UPDATE PERSONAL BEST OF PARTICLES IN THE POPULATION

import random
from math import *
import numpy
import copy


def update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL):
    summ=0
    for i in range(SWARM_SIZE):
        summ=0 
        if((PARTICLE_FIT[i][0]<pBestFitness[i][0]) and (PARTICLE_FIT[i][1]<pBestFitness[i][1]) and (PARTICLE_FIT[i][2]>pBestFitness[i][2])):
            summ+=no_of_obj_func
        elif((PARTICLE_FIT[i][0]>pBestFitness[i][0]) and (PARTICLE_FIT[i][1]>pBestFitness[i][1]) and (PARTICLE_FIT[i][2]<pBestFitness[i][2])):
            summ=0
        else:
            summ=1
        if(summ==no_of_obj_func):
            measure=0
        else:
            if(summ==0):
                measure=1
            else:
                measure=random.randint(0,1)
        if(measure==0):
            for j in range(no_of_obj_func):
                pBestFitness[i][j]=PARTICLE_FIT[i][j]
            for j in range(COUNT):
                pBestPosition[i][j]=PARTICLE_VAL[i][j]

In [ ]:
# FUNCTION TO UPDATE PERSONAL BEST OF PARTICLES IN THE POPULATION

import random
from math import *
import numpy
import copy


def update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL):
    summ=0
    for i in range(SWARM_SIZE):
        summ=0 
        if((PARTICLE_FIT[i][0]<pBestFitness[i][0]) and (PARTICLE_FIT[i][1]<pBestFitness[i][1]) and (PARTICLE_FIT[i][2]>pBestFitness[i][2])):
            summ+=no_of_obj_func
        elif((PARTICLE_FIT[i][0]>pBestFitness[i][0]) and (PARTICLE_FIT[i][1]>pBestFitness[i][1]) and (PARTICLE_FIT[i][2]<pBestFitness[i][2])):
            summ=0
        else:
            summ=1
        if(summ==no_of_obj_func):
            measure=0
        else:
            if(summ==0):
                measure=1
            else:
                measure=random.randint(0,1)
        if(measure==0):
            for j in range(no_of_obj_func):
                pBestFitness[i][j]=PARTICLE_FIT[i][j]
            for j in range(COUNT):
                pBestPosition[i][j]=PARTICLE_VAL[i][j]

In [ ]:
# FUNCTION TO CALCULATE WEIGHTED T SCORE FOR EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
import numpy

def weighted_t_score(Normal_Values,Tumor_Values,N_Value,T_Value,normal_count,tumor_count,GENE_IN_PARTICLE):
    
    DEG_OF_RELEVANCE=[random.uniform(0,1) for i in range(GENE_IN_PARTICLE)]
    #print DEG_OF_RELEVANCE
    for i in range(GENE_IN_PARTICLE):
        Normal_Values[i]=(Normal_Values[i]*DEG_OF_RELEVANCE[i])
    for i in range(GENE_IN_PARTICLE):
        Tumor_Values[i]=(Tumor_Values[i]*DEG_OF_RELEVANCE[i])
    
    FINAL_VALUE1=0.0
    FINAL_WEIGHT=0.0
    FINAL_VALUE2=0.0
    
    for i in range(GENE_IN_PARTICLE):
        FINAL_VALUE1+=Normal_Values[i]
        FINAL_WEIGHT+=DEG_OF_RELEVANCE[i]
    
                               
    for i in range(GENE_IN_PARTICLE):
        FINAL_VALUE2+=Tumor_Values[i]
           
    Mean1=FINAL_VALUE1/FINAL_WEIGHT # weighted mean of class 1
    Mean2=FINAL_VALUE2/FINAL_WEIGHT # weighted mean of class 2
   
    for i in range(GENE_IN_PARTICLE):
        N_Value[i]=(abs(N_Value[i]-Mean1)**2)*DEG_OF_RELEVANCE[i]
        
        
    for i in range(GENE_IN_PARTICLE):
        T_Value[i]=(abs(T_Value[i]-Mean2)**2)*DEG_OF_RELEVANCE[i]
        
    FINAL_STD1=0.0
    FINAL_STD2=0.0
    for i in range(GENE_IN_PARTICLE):
        FINAL_STD1+=N_Value[i]
                               
    for i in range(GENE_IN_PARTICLE):
        FINAL_STD2+=T_Value[i]
        
    
    StdDev1=(((normal_count/(normal_count-1))*(FINAL_STD1/FINAL_WEIGHT))**0.5) # weighted standard deviation of class 1
    StdDev2=(((tumor_count/(tumor_count-1))*(FINAL_STD2/FINAL_WEIGHT))**0.5) # weighted standard deviation of class 2
    
    WEIGHTED_T_SCORE=(abs(Mean1-Mean2))/(((StdDev1/normal_count)+(StdDev2/tumor_count))**0.5)
    return(WEIGHTED_T_SCORE)

def Weighted_T_Score(POPULATION_SIZE):
    
    no_of_pathways=0
    NO_OF_GENES = []
    NumberOfGenes=[]
    arr1=list()
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                NumberOfGenes.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
   

    NORMAL_DATA=[]
    TUMOR_DATA=[]
    n_count=0
    t_count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                n_count+=1
            elif(line.startswith("tumor")):
                t_count+=1
       
    COUNT=n_count+t_count
    ALL_GENES=[]
    Total=0
    VALUES=[]
    Normal_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    N_Value=[float(0.0) for i  in range(GENES_IN_PARTICLE)]
    Tumor_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    T_Value=[float(0.0)for i in range(GENES_IN_PARTICLE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    #print GENES           
    for i in range(no_of_pathways):
        for j in range(NumberOfGenes[i]):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                Normal_Values[g]+=float(VALUES[k])
                                N_Value[g]+=float(VALUES[k])
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                Tumor_Values[g]+=float(VALUES[k])
                                T_Value[g]+=float(VALUES[k])
            g+=1
            
    T_SCORE=[[] for i in range(POPULATION_SIZE)]
    for q in range(POPULATION_SIZE):
        #print ("Weighted T Score For Particle " +str(q+1)+" :")
        Norm_Values=copy.copy(Normal_Values)
        Tum_Values=copy.copy(Tumor_Values)
        N_Values=copy.copy(N_Value)
        T_Values=copy.copy(T_Value)
        
        T_SCORE[q]=weighted_t_score(Normal_Values,Tumor_Values,N_Value,T_Value,n_count,t_count,GENES_IN_PARTICLE)
        
        Normal_Values=copy.copy(Norm_Values)
        Tumor_Values=copy.copy(Tum_Values)
        N_Value=copy.copy(N_Values)
        T_Value=copy.copy(T_Values)
        
    return(T_SCORE)

In [ ]:
# FUNCTION TO CALCULATE WEIGHTED Z SCORE FOR EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
import numpy


def weighted_z_score(Exp_Values,GENES_IN_PARTICLE,Expression_Values,COUNT):
    WEIGHTED_RANDOM=[(random.random()) for i in range(GENES_IN_PARTICLE)]
    FINAL_WEIGHT=0.0
    for i in range(GENES_IN_PARTICLE):
        Exp_Values[i]=Exp_Values[i] * WEIGHTED_RANDOM[i]
    FINAL_P_ACT=0.0
    for i in range(GENES_IN_PARTICLE):
        FINAL_P_ACT+=Exp_Values[i]
        FINAL_WEIGHT+=WEIGHTED_RANDOM[i]
        
    PATHWAY_ACTIVITY=FINAL_P_ACT/(FINAL_WEIGHT**0.5)
    W_MEAN=FINAL_P_ACT/FINAL_WEIGHT 
    for i in range(GENES_IN_PARTICLE):
        Expression_Values[i]=(abs(Expression_Values[i]-W_MEAN)**2)*WEIGHTED_RANDOM[i]
    #print Expression_Values
    #print PATHWAY_ACTIVITY
    Variance=0.0
    for i in range(GENES_IN_PARTICLE):
        Variance+=Expression_Values[i]
        
    W_STD_DEV=(((COUNT/(COUNT-1))*(Variance/FINAL_WEIGHT))**0.5)
    WEIGHTED_Z_SCORE=(PATHWAY_ACTIVITY-W_MEAN)/W_STD_DEV
    return(WEIGHTED_Z_SCORE)


def Weighted_Z_Score(POPULATION_SIZE):
    no_of_pathways=0
    NO_OF_GENES = []
    NumberOfGenes=[]
    arr1=list()
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                NumberOfGenes.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
   

    NORMAL_DATA=[]
    TUMOR_DATA=[]
    n_count=0
    t_count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                n_count+=1
            elif(line.startswith("tumor")):
                t_count+=1
       
    COUNT=n_count+t_count
    
    ALL_GENES=[]
    Total=0
    VALUES=[]
    Normal_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    N_Value=[float(0.0) for i  in range(GENES_IN_PARTICLE)]
    Tumor_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    T_Value=[float(0.0)for i in range(GENES_IN_PARTICLE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    #print GENES           
    for i in range(no_of_pathways):
        for j in range(NumberOfGenes[i]):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                Normal_Values[g]+=float(VALUES[k])
                                N_Value[g]+=float(VALUES[k])
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                Tumor_Values[g]+=float(VALUES[k])
                                T_Value[g]+=float(VALUES[k])
            g+=1
    Exp_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    Expression_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    #print Normal_Values,"\n\n",Tumor_Values
    for i in range(GENES_IN_PARTICLE):
        Exp_Values[i]=Normal_Values[i]+Tumor_Values[i]
    Expression_Values=copy.copy(Exp_Values)
    #print len(Exp_Values)
    WEIGHTED_Z_SCORE=[[] for i in range(POPULATION_SIZE)]
    for p in range(POPULATION_SIZE):
        E1=copy.copy(Exp_Values)
        E2=copy.copy(Expression_Values)
        WEIGHTED_Z_SCORE[p]=weighted_z_score(Exp_Values,GENES_IN_PARTICLE,Expression_Values,COUNT)
        Exp_Values=copy.copy(E1)
        Expression_Values=copy.copy(E2) 
    return(WEIGHTED_Z_SCORE)

In [ ]:
# FUNCTION TO INSERT NONDOMINATED PARTICLES FROM POPULATION INTO ARCHIVE

import nbimporter
import random
from math import *
import copy
import numpy
import DELETE_PARTICLE
import CROWDING
reload(CROWDING)
reload(DELETE_PARTICLE)


def initialise_nondom_sol(no_of_nondom_sol,SWARM_SIZE,ARCHIVE_VAL,ARCHIVE_FIT,PARTICLE_VAL,PARTICLE_FIT,no_of_obj_func,archive_size,COUNT):
    flag=0
    for i in range(SWARM_SIZE):
        if(no_of_nondom_sol==0): # if archive is empty
            # insert nondominated solution from population into archive
            for j in range(COUNT):
                ARCHIVE_VAL[no_of_nondom_sol][j]=PARTICLE_VAL[i][j]
            for j in range(no_of_obj_func):
                ARCHIVE_FIT[no_of_nondom_sol][j]=PARTICLE_FIT[i][j]
            no_of_nondom_sol+=1
        else: # if archive is not empty
            flag=1
            for k in range(no_of_nondom_sol):
                total=0
                # check for non domination
                for j in range(no_of_obj_func):
                    if(((PARTICLE_FIT[i][j] < ARCHIVE_FIT[k][j]) and (j==0 | j==1)) or ((PARTICLE_FIT[i][j] > ARCHIVE_FIT[k][j]) and (j==2))):
                        total +=1;
                    if(total==no_of_obj_func): # if particle in population dominates
                    # delete particle in archive
                        no_of_nondom_sol=DELETE_PARTICLE.delete_particle(k,no_of_obj_func,no_of_nondom_sol,ARCHIVE_FIT)
                    elif(total==0): # particle in archive dominates
                        flag=0
                        break
            
            if(flag==1):
                if(no_of_nondom_sol < archive_size):# if archive memory is not full , insert particle
                    for j in range(COUNT):
                        ARCHIVE_VAL[no_of_nondom_sol][j]=PARTICLE_VAL[i][j]
                    for j in range(no_of_obj_func):
                        ARCHIVE_FIT[no_of_nondom_sol][j]=PARTICLE_FIT[i][j]
                    no_of_nondom_sol+=1
                else: # if archive memory is full , select particle to replace
                    # compute crowding distance values for particles in archive
                    no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
                    temp=(no_of_nondom_sol-1)*0.90
                    k=random.randint(int(temp),no_of_nondom_sol-1)
                    for j in range(COUNT):
                        ARCHIVE_VAL[k][j]=PARTICLE_VAL[i][j]
                    for j in range(no_of_obj_func):
                        ARCHIVE_FIT[k][j]=PARTICLE_FIT[i][j] 
            
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO INITIALISE POSITION OF PARTICLES IN POPULATION

import random
from math import *
import numpy

def initialise_PARTICLE_VALUES(COUNT,SWARM_SIZE):
    xmin=-5.12
    xmax=5.12
    initial=[[abs(xmin+random.uniform(0,1)*(xmax-xmin)) for j in range(COUNT)] for i in range(SWARM_SIZE)]
    return(initial)

In [ ]:
# FUNCTION TO INITIALISE VELOCITY OF PARTICLES IN POPULATION

import random
from math import *
import numpy


def initialise_velocity(COUNT,SWARM_SIZE):
    xmin=-5.12
    xmax=5.12
    vmin=0.25*xmin
    vmax=0.25*xmax
    init=[[(vmin+random.uniform(0,1)*(vmax-vmin)) for j in range(COUNT)] for i in range(SWARM_SIZE)]
    return(init)

In [ ]:
# FUNCTION TO CALCULATE SENSITIVITY , SPECIFICITY , ACCURACY , F_SCORE AND AREA UNDER CURVE VALUES OF EACH PARTICLE IN THE POPULATION


import random
from math import *
import numpy
import copy


#if __name__=='__main__':
def values(POPULATION_SIZE,NONDOM_PARTICLE,temp_nondom,VALUES_1):
    temp=1
    no_of_pathways=0
    NO_OF_GENES = []
    arr1=[]
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
    ALL_GENES=[]
    Total=0
    VALUES=[]
    tempSENSITIVITY=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    tempSPECIFICITY=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    accuracy=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    f_score=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    SENSITIVITY=[float(0.0) for i in range(POPULATION_SIZE)]
    SPECIFICITY=[float(0.0) for i in range(POPULATION_SIZE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    tp=0.0
    tn=0.0
    fp=0.0
    fn=0.0
    for i in range(no_of_pathways):
        for j in range(int(NO_OF_GENES[i])):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                if(VALUES[k]==str(0)):
                                    tn+=1.0
                                elif(VALUES[k]>str(0) and VALUES[k]<str(0.5)):
                                    fn+=1.0
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                if(VALUES[k]==str(1)):
                                    tp+=1.0
                                elif(VALUES[k]>str(0.5) and VALUES[k]<str(1.0)):
                                    fp+=1.0
                        tempSENSITIVITY[g]=(tp/(tp+fn))
                        tempSPECIFICITY[g]=(tn/(tn+fp))
                        accuracy[g]=(tp+tn)/(tp+tn+fp+fn)
                        f_score[g]=(2*tp)/((2*tp)+fp+fn)
                        tp=0.0
                        tn=0.0
                        fp=0.0
                        fn=0.0
                        g+=1
    
    
    
    Weighted_Score1=[float(0.0) for i in range(POPULATION_SIZE)]
    Weighted_Score2=[float(0.0) for i in range(POPULATION_SIZE)]
    Weighted_Score3=[float(0.0) for i in range(POPULATION_SIZE)]
    Weighted_Score4=[float(0.0) for i in range(POPULATION_SIZE)]
    
    SENSITIVITY_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    ACCURACY_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    F_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    
    MSENSITIVITY_SCORE=[int(1) for  i in range(POPULATION_SIZE)]
    # calculation of sensitivity of each particle in the population
    
    for  i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran1=random.random()
            SENSITIVITY_SCORE[i]+=(float(tempSENSITIVITY[j]*ran1))
            Weighted_Score1[i]+=ran1
            SENSITIVITY_SCORE[i]=(float(SENSITIVITY_SCORE[i])/float(Weighted_Score1[i]))
            MSENSITIVITY_SCORE[i]=MSENSITIVITY_SCORE[i]-SENSITIVITY_SCORE[i]
    SPECIFICITY_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    MSPECIFICITY_SCORE=[int(1) for i in range(POPULATION_SIZE)]
    # calculation of specificity of each particle in the population
    
    for i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran2=random.random()
            SPECIFICITY_SCORE[i]+=(float(tempSPECIFICITY[j]*ran2))
            Weighted_Score2[i]+=ran2
            SPECIFICITY_SCORE[i]=(float(SPECIFICITY_SCORE[i])/float(Weighted_Score2[i]))
            MSPECIFICITY_SCORE[i]=MSPECIFICITY_SCORE[i]-SENSITIVITY_SCORE[i]
   
    MACCURACY_SCORE=[int(1) for i in range(POPULATION_SIZE)]
    
    # calculation of accuracy score of each particle in the population
    for i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran3=random.random()
            ACCURACY_SCORE[i]+=(float(accuracy[j]*ran3))
            Weighted_Score3[i]+=ran3
            ACCURACY_SCORE[i]=(float(ACCURACY_SCORE[i])/float(Weighted_Score3[i]))
            MACCURACY_SCORE[i]=MACCURACY_SCORE[i]-ACCURACY_SCORE[i]

    MF_SCORE=[int(1) for i in range(POPULATION_SIZE)]
    
    # calculation of f score of each particle in the population
    for i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran4=random.random()
            F_SCORE[i]+=(float(f_score[j]*ran4))
            Weighted_Score4[i]+=ran4
            F_SCORE[i]=(float(F_SCORE[i])/float(Weighted_Score4[i]))
            MF_SCORE[i]=MF_SCORE[i]-F_SCORE[i]

    AREA_CURVE=[float(0.0) for i in range(POPULATION_SIZE)]
    MAREA_CURVE=[int(1) for i in range(POPULATION_SIZE)]
    
    # calculation of area under curve values of each particle in the population
    for i in range(POPULATION_SIZE):
        AREA_CURVE[i]=(SENSITIVITY_SCORE[i]/MSPECIFICITY_SCORE[i])
        MAREA_CURVE[i]=(MAREA_CURVE[i]-AREA_CURVE[i])
     
    for i in range(temp_nondom):
        VALUES_1[i][0]+=(MSENSITIVITY_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][1]+=(MSPECIFICITY_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][2]+=(MACCURACY_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][3]+=(MF_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][4]+=(MAREA_CURVE[int(NONDOM_PARTICLE[i])])

In [ ]:
# FUNCTION TO STORE PERSONAL BEST VALUES OF PARTICLES IN POPULATION 

import random
from math import *
import copy


def p_Best(pBestPosition,pBestFitness,PARTICLE_VAL,PARTICLE_FIT,SWARM_SIZE,no_of_obj_func,COUNT):
    for i in range(SWARM_SIZE): # store personal best positions of particles
        for j in range(COUNT):
            pBestPosition[i][j]=PARTICLE_VAL[i][j]
            
    for i in range(SWARM_SIZE): # store personal best fitness values of particles
        for j in range(no_of_obj_func):
            pBestFitness[i][j]=PARTICLE_FIT[i][j]

In [ ]:
#                  IMPLEMENTATION OF MULTI-OBJECTIVE PARTICLE SWARM OPTIMIZATION
#                  -------------------------------------------------------------                  
#
#
## This is the Main Function to execute the MOPSO Process


import nbimporter
import random
from math import *
import numpy
import copy
import init_VELOCITY as initvelo
import init_PARTICLE_VALUES as initparval
import PARTICLE_FITNESS
import store_pBest
import init_NONDOMINATED_SOLUTION
import CROWDING
import UPDATE_VELOCITY_POSITION
import UPDATE_ARCHIVE
import UPDATE_pBest
import sensitivity_specificity_accuracy_fscore_AUC
reload(sensitivity_specificity_accuracy_fscore_AUC)
reload(UPDATE_pBest)
reload(UPDATE_ARCHIVE)
reload(UPDATE_VELOCITY_POSITION)
reload(CROWDING)
reload(init_NONDOMINATED_SOLUTION)
reload(store_pBest)
reload(PARTICLE_FITNESS)
reload(initvelo)
reload(initparval)


# main function

if __name__=='__main__':
    normal_count=0         # Counts the number of Normal samples to be considered for individual gene
    tumor_count=0          # Counts the number of Tumor samples to be considered for individual gene
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                normal_count+=1
            elif(line.startswith("tumor")):
                tumor_count+=1
       
    COUNT=normal_count+tumor_count        # total number of samples for each gene
    SWARM_SIZE=25                         # total number of particles in the population
    no_of_obj_func=3                      # total number of objective functions considered
    Iteration=100
    archive_size=25                       # set maximum capacity of archive
    no_of_nondom_sol=0                    # Initial number of nondominated solutions in archive
    
    PARTICLE_VAL=initparval.initialise_PARTICLE_VALUES(COUNT,SWARM_SIZE) # initialise position values of particles in population 
    PARTICLE_FIT=[[[] for j in range(no_of_obj_func)] for i in range(SWARM_SIZE)]  # fitness values of particles in population
    
    ARCHIVE_VAL=[[[]for j in range(COUNT)] for i in range(archive_size)]  #position values of particles in archive
    ARCHIVE_FIT=[[[]for j in range(no_of_obj_func)]for i in range(archive_size)]  #fitness values of particles in archive
    
    VELOCITY=initvelo.initialise_velocity(COUNT,SWARM_SIZE)  # initialise velocity of particle in population
    
    pBestPosition=[[[]for j in range(COUNT)]for i in range(SWARM_SIZE)] # personal best position for each particle in population
    pBestFitness=[[[]for j in range(no_of_obj_func)]for i in range(SWARM_SIZE)]  # personal fitness for each particle in population
    
#     PARTICLE_FITNESS.update_particle(PARTICLE_FIT,no_of_obj_func,SWARM_SIZE) # update fitness values of each particle in population
    
    store_pBest.p_Best(pBestPosition,pBestFitness,PARTICLE_VAL,PARTICLE_FIT,SWARM_SIZE,no_of_obj_func,COUNT) # Store initial personal bests (both variable and fitness values) of particles
   
    
    # the output is written in the file MOPSO_OUTPUT.txt
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/MOPSO_OUTPUT.txt","w+") as am:
        for t in range(Iteration):
            # Compute new fitness values for each particle in the population
            PARTICLE_FITNESS.update_particle(PARTICLE_FIT,no_of_obj_func,SWARM_SIZE)
            # insert nondominated particles in archive from population
            no_of_nondom_sol=init_NONDOMINATED_SOLUTION.initialise_nondom_sol(no_of_nondom_sol,SWARM_SIZE,ARCHIVE_VAL,ARCHIVE_FIT,PARTICLE_VAL,PARTICLE_FIT,no_of_obj_func,archive_size,COUNT)
            
            if(no_of_nondom_sol>2):
                no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
            # Compute new velocity and position of each particle in the population
            no_of_nondom_sol=UPDATE_VELOCITY_POSITION.update_velocity_position(no_of_nondom_sol,SWARM_SIZE,COUNT,VELOCITY,pBestPosition,PARTICLE_VAL,ARCHIVE_VAL)

            # insert new non dominated particles in archive from population
            no_of_nondom_sol=UPDATE_ARCHIVE.update_archive(SWARM_SIZE,no_of_nondom_sol,archive_size,COUNT,ARCHIVE_VAL,PARTICLE_VAL,ARCHIVE_FIT,PARTICLE_FIT,no_of_obj_func)
            
            # update personal best for each particle in population
            UPDATE_pBest.update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL)
            
            NONDOM_PARTICLE=[int(0) for i in range(no_of_nondom_sol)]
            
            print (str(t))
            am.write("SIZE OF PARETO SET : ")
            am.write(str(no_of_nondom_sol))
            am.write("\n\n")
            am.write("ELEMENTS OF PARETO SET : ")
            for  i in range(no_of_nondom_sol):
                for j in range(SWARM_SIZE):
                    if(ARCHIVE_FIT[i][0]==PARTICLE_FIT[j][0]):
                        NONDOM_PARTICLE[i]=j
                        am.write(str(j+1))
                am.write("\t")
            am.write("\n\n")
                    
            am.write("FITNESS VALUES OF PARTICLES IN THE ARCHIVE : ")
            am.write("\n\n")
            am.write("WEIGHTED T SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][0]))
                am.write("\t\t")
            am.write("\n\n")
            am.write("WEIGHTED Z SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][1]))
                am.write("\t\t")
            am.write("\n\n")
            am.write("INTERACTION SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][2]))
                am.write("\t\t")
            am.write("\n")
            am.write("\n")
            am.write("**********************************************************************************************")    
            am.write("\n\n")
            
            temp_nondom=no_of_nondom_sol
            # reset number of nondominated particles, archive values and fitnesses to store values for next iteration
            no_of_nondom_sol=0
            
            ARCHIVE_VAL=[[[]for j in range(COUNT)] for i in range(archive_size)]
            ARCHIVE_FIT=[[[]for j in range(no_of_obj_func)]for i in range(archive_size)]
    
    # store the sensitivity , specificity , accuracy , f_score and area under curve value of the final non-dominated particles on archive
    VALUES_1=[[float(0.0) for j in range(5)] for i in range(temp_nondom)]
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/Output.txt" , "w+") as sn:
        sensitivity_specificity_accuracy_fscore_AUC.values(SWARM_SIZE,NONDOM_PARTICLE,temp_nondom,VALUES_1) # calculation of the corresponding values

        sn.write("ELEMENTS OF PARETO SET : ")
        for i in range(temp_nondom):
            sn.write(str(NONDOM_PARTICLE[i]+1))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("SENSITIVITY SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][0]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("SPECIFICITY SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][1]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("ACCURACY MEASURE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][2]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("F SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][3]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("AREA UNDER CURVE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][4]))
            sn.write("\t")
        sn.write("\n\n")

# PARTICLE SWARM OPTIMIZATION CODE2

In [ ]:
import random
import math
import matplotlib.pyplot as plt
#------------------------------------------------------------------------------
# TO CUSTOMIZE THIS PSO CODE TO SOLVE UNCONSTRAINED OPTIMIZATION PROBLEMS, CHANGE THE PARAMETERS IN THIS SECTION ONLY:
# THE FOLLOWING PARAMETERS MUST BE CHANGED.
def objective_function(x):
    y = 3*(1-x[0])**2*math.exp(-x[0]**2 - (x[1]+1)**2) - 10*(x[0]/5 - x[0]**3 - x[1]**5)*math.exp(-x[0]**2 - x[1]**2) -1/3*math.exp(-(x[0]+1)**2 - x[1]**2);
    return y
 
bounds=[(-3,3),(-3,3)]   # upper and lower bounds of variables
nv = 2                   # number of variables
mm = -1                   # if minimization problem, mm = -1; if maximization problem, mm = 1
 
# THE FOLLOWING PARAMETERS ARE OPTINAL.
particle_size=100         # number of particles
iterations=200           # max number of iterations
w=0.85                    # inertia constant
c1=1                    # cognative constant
c2=2                     # social constant
# END OF THE CUSTOMIZATION SECTION
#------------------------------------------------------------------------------    
class Particle:
    def __init__(self,bounds):
        self.particle_position=[]                     # particle position
        self.particle_velocity=[]                     # particle velocity
        self.local_best_particle_position=[]          # best position of the particle
        self.fitness_local_best_particle_position= initial_fitness  # initial objective function value of the best particle position
        self.fitness_particle_position=initial_fitness             # objective function value of the particle position
 
        for i in range(nv):
            self.particle_position.append(random.uniform(bounds[i][0],bounds[i][1])) # generate random initial position
            self.particle_velocity.append(random.uniform(-1,1)) # generate random initial velocity
 
    def evaluate(self,objective_function):
        self.fitness_particle_position=objective_function(self.particle_position)
        if mm == -1:
            if self.fitness_particle_position < self.fitness_local_best_particle_position:
                self.local_best_particle_position=self.particle_position                  # update the local best
                self.fitness_local_best_particle_position=self.fitness_particle_position  # update the fitness of the local best
        if mm == 1:
            if self.fitness_particle_position > self.fitness_local_best_particle_position:
                self.local_best_particle_position=self.particle_position                  # update the local best
                self.fitness_local_best_particle_position=self.fitness_particle_position  # update the fitness of the local best
 
    def update_velocity(self,global_best_particle_position):
        for i in range(nv):
            r1=random.random()
            r2=random.random()
 
            cognitive_velocity = c1*r1*(self.local_best_particle_position[i] - self.particle_position[i])
            social_velocity = c2*r2*(global_best_particle_position[i] - self.particle_position[i])
            self.particle_velocity[i] = w*self.particle_velocity[i]+ cognitive_velocity + social_velocity
 
    def update_position(self,bounds):
        for i in range(nv):
            self.particle_position[i]=self.particle_position[i]+self.particle_velocity[i]
 
            # check and repair to satisfy the upper bounds
            if self.particle_position[i]>bounds[i][1]:
                self.particle_position[i]=bounds[i][1]
            # check and repair to satisfy the lower bounds
            if self.particle_position[i] < bounds[i][0]:
                self.particle_position[i]=bounds[i][0]
                 
class PSO():
    def __init__(self,objective_function,bounds,particle_size,iterations):
 
        fitness_global_best_particle_position=initial_fitness
        global_best_particle_position=[]
 
        swarm_particle=[]
        for i in range(particle_size):
            swarm_particle.append(Particle(bounds))
        A=[]
         
        for i in range(iterations):
            for j in range(particle_size):
                swarm_particle[j].evaluate(objective_function)
 
                if mm ==-1:
                    if swarm_particle[j].fitness_particle_position < fitness_global_best_particle_position:
                        global_best_particle_position = list(swarm_particle[j].particle_position)
                        fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
                if mm ==1:
                    if swarm_particle[j].fitness_particle_position > fitness_global_best_particle_position:
                        global_best_particle_position = list(swarm_particle[j].particle_position)
                        fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
            for j in range(particle_size):
                swarm_particle[j].update_velocity(global_best_particle_position)
                swarm_particle[j].update_position(bounds)
                 
            A.append(fitness_global_best_particle_position) # record the best fitness
             
             
        print('Optimal solution:', global_best_particle_position)
        print('Objective function value:', fitness_global_best_particle_position)
        print('Evolutionary process of the objective function value:')
        plt.plot(A)
#------------------------------------------------------------------------------
if mm == -1:
    initial_fitness = float("inf") # for minimization problem
if mm == 1:
    initial_fitness = -float("inf") # for maximization problem
#------------------------------------------------------------------------------   
# Main PSO         
PSO(objective_function,bounds,particle_size,iterations)